# Import

In [15]:
import os
import sys
sys.path.append(os.path.join(os.getcwd(),".."))

import time
import math
import json
import glob

import numpy as np
import pandas as pd
import scipy
from scipy.optimize import minimize, least_squares
import matplotlib.pyplot as plt

import cv2
import torch
from torch.nn.functional import mse_loss

import pytorch_kinematics as pk
from urdf_parser_py.urdf import URDF

In [2]:
# Explicitly telling to use GPU
torch.set_default_device('cuda')
torch.set_default_dtype(d=torch.float64)

# Helper Functions

In [3]:
def load_json(filename: str):
    # Construct the absolute path by joining the current directory and relative path
    absolute_path = os.path.join(os.getcwd(), '..', filename)
    # Load the JSON data
    with open(absolute_path, 'r') as file:
        json_data = json.load(file)    
    return json_data

def projectPoints(pts_3d, ext_mtx, cam_mtx, dist_coef):
    # Apply extrinsic transformation (rotation + translation)
    pts_3d_transformed = (ext_mtx @ pts_3d).reshape(-1,4).T
    
    # Convert back from homogeneous to 2D coordinates
    pts_2d_prime = pts_3d_transformed[:2, :] / pts_3d_transformed[2, :]
    x_prime = pts_2d_prime[0]
    y_prime = pts_2d_prime[1]
    
    # Distortion Parameters Calculation
    k1, k2, p1, p2, k3 = dist_coef
    
    r2 = x_prime**2 + y_prime**2
    r4 = r2**2
    r6 = r2*r4
    a1 = 2*x_prime*y_prime
    a2 = r2 + 2*x_prime**2
    a3 = r2 + 2*y_prime**2
    
    # Radial distortion
    radial_distortion = 1 + k1*r2 + k2*r4 + k3*r6
    
    # Tangential distortion
    tangential_distortion_x = p1*a1 + p2*a2
    tangential_distortion_y = p1*a3 + p2*a1
    
    # Apply distortion
    x_double_prime = x_prime*radial_distortion + tangential_distortion_x
    y_double_prime = y_prime* radial_distortion + tangential_distortion_y
    
    # Intrinsic Camera Matrix
    fx = cam_mtx[0,0]
    cx = cam_mtx[0,2]
    fy = cam_mtx[1,1]
    cy = cam_mtx[1,2]
    
    u = x_double_prime*fx + cx
    v = y_double_prime*fy + cy

    return u,v

def xml_to_str(robot):
    # To XML string with filtering
    temp_str = robot.to_xml_string()
    words = temp_str.split()
    words[5] = '>'
    urdf_str = ' '.join(words)
    return urdf_str

# Initialization

In [4]:
# PyTorch Configs

device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float64

In [5]:
# Load Configs
cam_mtxs = load_json('camera_mtx.json')
gaze_ctrs = load_json('calib_params.json')
urdf_path = os.path.join(os.getcwd(),"..","urdf","chest_grace.urdf")
robot = URDF.from_xml_file(urdf_path)

In [6]:
# Load Dataset CSV

data_dir = "final"
csv_file = os.path.join(os.getcwd(),"..","data", data_dir, "240909_grace_dataset.csv")
df = pd.read_csv(csv_file)
temp_df = df.iloc[:,:12]
print('Total:',len(temp_df))
temp_df

Total: 209731


,x_c_l,y_c_l,z_c_l,x_c_r,y_c_r,z_c_r,cmd_theta_lower_neck_pan,cmd_theta_lower_neck_tilt,cmd_theta_upper_neck_tilt,cmd_theta_left_eye_pan,cmd_theta_right_eye_pan,cmd_theta_eyes_tilt
0,-0.873669,-0.334702,0.738088,-1.166921,-0.352033,0.738508,-35,-10,-10,-14,-14,20
1,-0.821681,-0.331576,0.738000,-0.997318,-0.340332,0.738221,-35,-10,-10,-12,-12,20
2,-0.731501,-0.324165,0.738176,-0.868484,-0.332085,0.738360,-35,-10,-10,-10,-10,20
3,-0.672994,-0.320809,0.737795,-0.776648,-0.327031,0.737932,-35,-10,-10,-8,-8,20
4,-0.613507,-0.317492,0.737656,-0.690487,-0.321762,0.737752,-35,-10,-10,-6,-6,20
...,...,...,...,...,...,...,...,...,...,...,...,...
209726,0.792502,-0.267949,0.950136,0.716083,-0.253872,0.953027,35,30,40,4,4,-30
209727,0.871620,-0.284761,0.947121,0.792232,-0.271009,0.950140,35,30,40,6,6,-30
209728,0.958315,-0.302900,0.943909,0.875639,-0.289312,0.947054,35,30,40,8,8,-30
209729,1.054044,-0.323434,0.940457,0.967562,-0.310196,0.943755,35,30,40,10,10,-30


In [7]:
# Sample 1000
small_df = temp_df.sample(n=1000, random_state=1).reset_index(drop=True)

In [8]:
# Convert DF Degrees Column to Radians

data_df = small_df.copy()
data_df['cmd_theta_lower_neck_pan'] = np.radians(data_df['cmd_theta_lower_neck_pan'].values)
data_df['cmd_theta_lower_neck_tilt'] = np.radians(data_df['cmd_theta_lower_neck_tilt'].values)
data_df['cmd_theta_upper_neck_tilt'] = np.radians(data_df['cmd_theta_upper_neck_tilt'].values)
data_df['cmd_theta_left_eye_pan'] = np.radians(data_df['cmd_theta_left_eye_pan'].values)
data_df['cmd_theta_right_eye_pan'] = np.radians(data_df['cmd_theta_right_eye_pan'].values)
data_df['cmd_theta_eyes_tilt'] = np.radians(data_df['cmd_theta_eyes_tilt'].values)
data_df

,x_c_l,y_c_l,z_c_l,x_c_r,y_c_r,z_c_r,cmd_theta_lower_neck_pan,cmd_theta_lower_neck_tilt,cmd_theta_upper_neck_tilt,cmd_theta_left_eye_pan,cmd_theta_right_eye_pan,cmd_theta_eyes_tilt
0,0.375717,-0.379865,0.994915,0.317935,-0.374271,0.995427,0.261799,0.523599,0.523599,0.034907,0.034907,0.174533
1,0.190400,-0.774233,0.983755,0.079054,-1.068046,0.992794,0.436332,0.000000,0.698132,-0.174533,-0.174533,0.349066
2,0.110060,-0.402567,0.978923,0.073115,-0.409601,0.980520,-0.174533,0.349066,0.523599,0.139626,0.139626,0.174533
3,0.266975,-0.505674,1.211371,0.204180,-0.511531,1.213977,-0.087266,0.523599,0.698132,0.139626,0.139626,0.261799
4,1.067655,-0.267191,1.175870,0.945559,-0.258172,1.180878,0.523599,0.174533,0.174533,0.104720,0.104720,-0.174533
...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.057498,-0.429063,1.217779,-0.044983,-0.415550,1.220390,0.174533,0.174533,0.698132,-0.139626,-0.139626,-0.349066
996,0.172112,-0.625225,0.741182,0.131402,-0.622980,0.741134,0.174533,0.000000,0.698132,0.000000,0.000000,0.174533
997,-0.572435,-0.565999,1.235573,-0.725942,-0.581753,1.240015,-0.349066,0.174533,0.698132,-0.139626,-0.139626,-0.174533
998,0.234568,-0.652529,1.215793,0.157167,-0.658971,1.219176,-0.087266,-0.174533,0.349066,0.104720,0.104720,0.087266


In [9]:
# Decision Variables Initial Value

var_dict= {
    'neck_pitch_polyfit_b0': 0.1,
    'neck_pitch_polyfit_b1': 0.1,
    'neck_pitch_polyfit_b2': 0,
    'neck_pitch_polyfit_b3': 0,

    'neck_yaw_polyfit_b0': 0.1,
    'neck_yaw_polyfit_b1': 0.1,
    'neck_yaw_polyfit_b2': 0,
    'neck_yaw_polyfit_b3': 0,

    'head_pitch_polyfit_b0': 0.1,
    'head_pitch_polyfit_b1': 0.1,
    'head_pitch_polyfit_b2': 0,
    'head_pitch_polyfit_b3': 0,

    'eyes_pitch_polyfit_b0': 0.1,
    'eyes_pitch_polyfit_b1': 0.1,
    'eyes_pitch_polyfit_b2': 0,
    'eyes_pitch_polyfit_b3': 0,

    'lefteye_yaw_polyfit_b0': 0.1,
    'lefteye_yaw_polyfit_b1': 0.1,
    'lefteye_yaw_polyfit_b2': 0,
    'lefteye_yaw_polyfit_b3': 0,

    'righteye_yaw_polyfit_b0': 0.1,
    'righteye_yaw_polyfit_b1': 0.1,
    'righteye_yaw_polyfit_b2': 0,
    'righteye_yaw_polyfit_b3': 0,

    'torso_origin_x': 0.12234,
    'torso_origin_y': -0.0325,
    'torso_origin_z': 0.05692,

    'neck_pitch_rot_r': 0,
    
    'head_pitch_origin_z': 0.13172,
    'head_pitch_rot_r': 0,
    'head_pitch_rot_y': 0,

    'eyes_pitch_origin_x': 0.08492,
    'eyes_pitch_origin_z': 0.05186,
    'eyes_pitch_rot_r': 0,
    'eyes_pitch_rot_y': 0,

    'lefteye_yaw_origin_y': 0.02895,
    
    'righteye_yaw_origin_y': -0.02895,

    'lefteye_cam_origin_x': 0.015,
    'lefteye_cam_rot_r': 0,
    'lefteye_cam_rot_p': 0,
    'lefteye_cam_rot_y': 0,

    'righteye_cam_origin_x': 0.015,
    'righteye_cam_rot_r': 0,
    'righteye_cam_rot_p': 0,
    'righteye_cam_rot_y': 0,
}

# Decision Variables 
var_list = list(var_dict.values())

# Variable Names
var_names_list = list(var_dict.keys())

In [10]:
# Index to Name Mapping
var2idx = dict(zip(list(range(len(var_names_list))),var_names_list))
var2idx

{0: 'neck_pitch_polyfit_b0',
 1: 'neck_pitch_polyfit_b1',
 2: 'neck_pitch_polyfit_b2',
 3: 'neck_pitch_polyfit_b3',
 4: 'neck_yaw_polyfit_b0',
 5: 'neck_yaw_polyfit_b1',
 6: 'neck_yaw_polyfit_b2',
 7: 'neck_yaw_polyfit_b3',
 8: 'head_pitch_polyfit_b0',
 9: 'head_pitch_polyfit_b1',
 10: 'head_pitch_polyfit_b2',
 11: 'head_pitch_polyfit_b3',
 12: 'eyes_pitch_polyfit_b0',
 13: 'eyes_pitch_polyfit_b1',
 14: 'eyes_pitch_polyfit_b2',
 15: 'eyes_pitch_polyfit_b3',
 16: 'lefteye_yaw_polyfit_b0',
 17: 'lefteye_yaw_polyfit_b1',
 18: 'lefteye_yaw_polyfit_b2',
 19: 'lefteye_yaw_polyfit_b3',
 20: 'righteye_yaw_polyfit_b0',
 21: 'righteye_yaw_polyfit_b1',
 22: 'righteye_yaw_polyfit_b2',
 23: 'righteye_yaw_polyfit_b3',
 24: 'torso_origin_x',
 25: 'torso_origin_y',
 26: 'torso_origin_z',
 27: 'neck_pitch_rot_r',
 28: 'head_pitch_origin_z',
 29: 'head_pitch_rot_r',
 30: 'head_pitch_rot_y',
 31: 'eyes_pitch_origin_x',
 32: 'eyes_pitch_origin_z',
 33: 'eyes_pitch_rot_r',
 34: 'eyes_pitch_rot_y',
 35: 'le

In [11]:
# Tensor Initialization

# Chest Camera OpenCV to World Orientation (LHR > RHR)
T_opencv_world = torch.Tensor([[0.0, 0.0, 1.0, 0.0],
                               [-1.0, 0.0, 0.0, 0.0],
                               [0.0, -1.0, 0.0, 0.0],
                               [0.0, 0.0, 0.0, 1.0]]).to(dtype=dtype, device=device)

# World Orientation to Chest Camera OpenCV (RHR > LHR)
T_world_opencv = torch.Tensor([[0.0, -1.0, 0.0, 0.0],
                               [0.0, 0.0, -1.0, 0.0],
                               [1.0, 0.0, 0.0, 0.0],
                               [0.0, 0.0, 0.0, 1.0]]).to(dtype=dtype, device=device)

### Camera Projection
# Camera Parameters
left_cam_mtx = torch.Tensor(cam_mtxs['left_eye']['camera_matrix'])
left_dist_coef = torch.Tensor(cam_mtxs['left_eye']['distortion_coefficients']).squeeze()
right_cam_mtx = torch.Tensor(cam_mtxs['right_eye']['camera_matrix'])
right_dist_coef = torch.Tensor(cam_mtxs['right_eye']['distortion_coefficients']).squeeze()

# [Test] Numpy Input

In [22]:
# Input
V = np.array(var_list)
x_c_l = torch.Tensor(data_df['x_c_l']).reshape(-1,1).to(dtype=dtype, device=device)
y_c_l = torch.Tensor(data_df['y_c_l']).reshape(-1,1).to(dtype=dtype, device=device)
z_c_l = torch.Tensor(data_df['z_c_l']).reshape(-1,1).to(dtype=dtype, device=device)
x_c_r = torch.Tensor(data_df['x_c_r']).reshape(-1,1).to(dtype=dtype, device=device)
y_c_r = torch.Tensor(data_df['y_c_r']).reshape(-1,1).to(dtype=dtype, device=device)
z_c_r = torch.Tensor(data_df['z_c_r']).reshape(-1,1).to(dtype=dtype, device=device)
cmd_lnt = torch.Tensor(data_df['cmd_theta_lower_neck_tilt']).reshape(-1,1).to(dtype=dtype, device=device)
cmd_lnp = torch.Tensor(data_df['cmd_theta_lower_neck_pan']).reshape(-1,1).to(dtype=dtype, device=device)
cmd_unt = torch.Tensor(data_df['cmd_theta_upper_neck_tilt']).reshape(-1,1).to(dtype=dtype, device=device)
cmd_et = torch.Tensor(data_df['cmd_theta_eyes_tilt']).reshape(-1,1).to(dtype=dtype, device=device)
cmd_lep = torch.Tensor(data_df['cmd_theta_left_eye_pan']).reshape(-1,1).to(dtype=dtype, device=device)
cmd_rep = torch.Tensor(data_df['cmd_theta_right_eye_pan']).reshape(-1,1).to(dtype=dtype, device=device)

In [19]:
# Input
V = np.array(var_list)
x_c_l = torch.Tensor([data_df['x_c_l'][0]]).reshape(-1,1).to(dtype=dtype, device=device)
y_c_l = torch.Tensor([data_df['y_c_l'][0]]).reshape(-1,1).to(dtype=dtype, device=device)
z_c_l = torch.Tensor([data_df['z_c_l'][0]]).reshape(-1,1).to(dtype=dtype, device=device)
x_c_r = torch.Tensor([data_df['x_c_r'][0]]).reshape(-1,1).to(dtype=dtype, device=device)
y_c_r = torch.Tensor([data_df['y_c_r'][0]]).reshape(-1,1).to(dtype=dtype, device=device)
z_c_r = torch.Tensor([data_df['z_c_r'][0]]).reshape(-1,1).to(dtype=dtype, device=device)
cmd_lnt = torch.Tensor([data_df['cmd_theta_lower_neck_tilt'][0]]).reshape(-1,1).to(dtype=dtype, device=device)
cmd_lnp = torch.Tensor([data_df['cmd_theta_lower_neck_pan'][0]]).reshape(-1,1).to(dtype=dtype, device=device)
cmd_unt = torch.Tensor([data_df['cmd_theta_upper_neck_tilt'][0]]).reshape(-1,1).to(dtype=dtype, device=device)
cmd_et = torch.Tensor([data_df['cmd_theta_eyes_tilt'][0]]).reshape(-1,1).to(dtype=dtype, device=device)
cmd_lep = torch.Tensor([data_df['cmd_theta_left_eye_pan'][0]]).reshape(-1,1).to(dtype=dtype, device=device)
cmd_rep = torch.Tensor([data_df['cmd_theta_right_eye_pan'][0]]).reshape(-1,1).to(dtype=dtype, device=device)

In [23]:
V = var_list

start = time.time()

### Joint-to-Motor Polynomial Models

neck_pitch = (torch.cat((torch.ones(cmd_lnt.shape[0], 1), cmd_lnt, cmd_lnt.pow(2), cmd_lnt.pow(3)), dim=1) @ torch.Tensor([[V[0]],[V[1]],[V[2]],[V[3]]]).to(dtype=dtype, device=device))
neck_yaw = (torch.cat((torch.ones(cmd_lnp.shape[0], 1), cmd_lnp, cmd_lnp.pow(2), cmd_lnp.pow(3)), dim=1) @ torch.Tensor([[V[4]],[V[5]],[V[6]],[V[7]]]).to(dtype=dtype, device=device))
head_pitch = (torch.cat((torch.ones(cmd_unt.shape[0], 1), cmd_unt, cmd_unt.pow(2), cmd_unt.pow(3)), dim=1) @ torch.Tensor([[V[8]],[V[9]],[V[10]],[V[11]]]).to(dtype=dtype, device=device))
eyes_pitch = (torch.cat((torch.ones(cmd_et.shape[0], 1), cmd_et, cmd_et.pow(2), cmd_et.pow(3)), dim=1) @ torch.Tensor([[V[12]],[V[13]],[V[14]],[V[15]]]).to(dtype=dtype, device=device))
lefteye_yaw = (torch.cat((torch.ones(cmd_lep.shape[0], 1), cmd_lep, cmd_lep.pow(2), cmd_lep.pow(3)), dim=1) @ torch.Tensor([[V[16]],[V[17]],[V[18]],[V[19]]]).to(dtype=dtype, device=device))
righteye_yaw = (torch.cat((torch.ones(cmd_rep.shape[0], 1), cmd_rep, cmd_rep.pow(2), cmd_rep.pow(3)), dim=1) @ torch.Tensor([[V[20]],[V[21]],[V[22]],[V[23]]]).to(dtype=dtype, device=device))


### URDF
# Variable Assignment
for joint in robot.joints:
    if joint.name == 'torso':
        joint.origin.position[0] = V[24]
        joint.origin.position[1] = V[25]
        joint.origin.position[2] = V[26]
    elif joint.name == 'neck_pitch':
        joint.origin.rotation[0] = V[27]
    elif joint.name == 'head_pitch':
        joint.origin.position[2] = V[28]
        joint.origin.rotation[0] = V[29]
        joint.origin.rotation[2] = V[30]
    elif joint.name == 'eyes_pitch':
        joint.origin.position[0] = V[31]
        joint.origin.position[2] = V[32]
        joint.origin.rotation[0] = V[33]
        joint.origin.rotation[2] = V[34]
    elif joint.name == 'lefteye_yaw':
        joint.origin.position[1] = V[35]
    elif joint.name == 'righteye_yaw':
        joint.origin.position[1] = V[36]
    elif joint.name == 'lefteye_cam':
        joint.origin.position[0] = V[37]
        joint.origin.rotation[0] = V[38]
        joint.origin.rotation[1] = V[39]
        joint.origin.rotation[2] = V[40]
    elif joint.name == 'righteye_cam':
        joint.origin.position[0] = V[41]
        joint.origin.rotation[0] = V[42]
        joint.origin.rotation[1] = V[43]
        joint.origin.rotation[2] = V[44]

# XML to String
urdf_str = xml_to_str(robot)

# Kinematic Chain
chain = pk.build_chain_from_urdf(urdf_str)
chain.to(dtype=dtype, device=device)

# Specifying Joint Angles (radians)
joint_cmd = torch.cat((neck_pitch, neck_yaw, head_pitch, eyes_pitch, lefteye_yaw, righteye_yaw), dim=1)

# Forward Kinematics
ret = chain.forward_kinematics(joint_cmd)



# Realsense to Left Eye Camera (Pytorch) with Points
T_clprime = torch.linalg.inv(ret['realsense'].get_matrix()) @ ret['leftcamera'].get_matrix()

# Realsense to Right Eye Camera (Pytorch) with Points
T_crprime = torch.linalg.inv(ret['realsense'].get_matrix()) @ ret['rightcamera'].get_matrix()

# Extrinsic Matrix
T_left_ext = T_world_opencv @ T_clprime @ T_opencv_world
T_right_ext = T_world_opencv @ T_crprime @ T_opencv_world


# Chest Camera OpenCV Orientation 3D Projection Points
left_chest_pts = torch.cat((x_c_l, y_c_l, z_c_l,torch.ones(x_c_l.shape[0], 1)), dim=1).reshape(-1,4,1)
right_chest_pts = torch.cat((x_c_r, y_c_r, z_c_r,torch.ones(x_c_r.shape[0], 1)), dim=1).reshape(-1,4,1)

# Points Projection
u_left, v_left = projectPoints(left_chest_pts, T_left_ext, left_cam_mtx, left_dist_coef)   
u_right, v_right = projectPoints(right_chest_pts, T_right_ext, right_cam_mtx, right_dist_coef)

# Clamping
u_min_val = 0
u_max_val = 639 
v_min_val = 0
v_max_val = 479
# u_left = torch.clamp(u_left, min=u_min_val, max=u_max_val)
# v_left = torch.clamp(v_left, min=v_min_val, max=v_max_val)
# u_right = torch.clamp(u_right, min=u_min_val, max=u_max_val)
# v_right = torch.clamp(v_right, min=v_min_val, max=v_max_val)

# Scaling
# u_left = min_max_scaler(u_left, u_min_val, u_max_val)
# v_left = min_max_scaler(v_left, v_min_val, v_max_val)
# u_right = min_max_scaler(u_right, u_min_val, u_max_val)
# v_right = min_max_scaler(v_right, v_min_val, v_max_val)

# True Value
true_u_left = gaze_ctrs['left_eye']['x_center']
true_v_left = gaze_ctrs['left_eye']['y_center']
true_u_right = gaze_ctrs['right_eye']['x_center']
true_v_right = gaze_ctrs['right_eye']['y_center']
# true_u_left = min_max_scaler(gaze_ctrs['left_eye']['x_center'], u_min_val, u_max_val)
# true_v_left = min_max_scaler(gaze_ctrs['left_eye']['y_center'], v_min_val, v_max_val)
# true_u_right = min_max_scaler(gaze_ctrs['right_eye']['x_center'], u_min_val, u_max_val)
# true_v_right = min_max_scaler(gaze_ctrs['right_eye']['y_center'], v_min_val, v_max_val)

# Loss
loss = (torch.sum((u_left-true_u_left)**2 + (v_left-true_v_left)**2 
        + (u_right-true_u_right)**2 + (v_right-true_v_right)**2)/u_left.shape[0])

end = time.time()
print('Elapsed Time: %.f ms' % (1000*(end-start)))
print(loss)
print(u_left.shape, v_left.shape)
print(u_right.shape, v_right.shape)

Elapsed Time: 11 ms
tensor(8.6195e+11, device='cuda:0')
torch.Size([1000]) torch.Size([1000])
torch.Size([1000]) torch.Size([1000])


In [24]:
residuals = torch.cat(((u_left-true_u_left).reshape(-1,1),(v_left-true_v_left).reshape(-1,1),
          (u_right-true_u_right).reshape(-1,1),(v_right-true_v_right).reshape(-1,1)), dim=1).cpu().numpy()
print(residuals.shape)
residuals

(1000, 4)


array([[ 6.01728234e+02, -1.22889758e+03,  9.79373013e+02,
        -1.43706517e+03],
       [ 3.28372900e+03, -1.20035935e+04,  2.89941877e+04,
        -1.46015707e+05],
       [ 1.01930083e+02, -7.49101485e+02,  4.15392528e+02,
        -8.94861521e+02],
       ...,
       [-2.96742590e+02, -6.33091566e+02, -1.09850850e+02,
        -7.29443028e+02],
       [ 1.95427330e+02, -8.50720327e+02,  4.82639027e+02,
        -9.84271290e+02],
       [ 1.94682532e+02, -5.17262633e+02,  4.73535437e+02,
        -5.59183753e+02]])

In [29]:
res_sum = np.sum(residuals, axis=1)
print(res_sum.shape)
res_sum

(1000,)


array([-1.08486150e+03, -1.25741384e+05, -1.12664040e+03, -1.51669514e+03,
        2.97465678e+03, -4.90070152e+02, -1.71654909e+03, -1.41960215e+02,
       -1.24544292e+03, -1.01025342e+03, -7.63595710e+02,  3.80876493e+02,
       -2.43649136e+04, -5.29582200e+02, -6.05910583e+02, -1.54421272e+03,
       -5.25724737e+02, -1.31320837e+03, -2.98135049e+02, -2.04729498e+02,
       -1.74388918e+04, -5.79856738e+02, -4.33598657e+03, -2.86912430e+03,
       -3.62537404e+03, -1.05561698e+03, -1.52276681e+03, -7.27690426e+02,
       -1.42825339e+03, -1.38634470e+03, -1.73136327e+03,  1.28379326e+03,
       -1.69045752e+02, -8.35826772e+02, -4.94382546e+02, -7.25348270e+02,
       -1.47991915e+03, -3.20844036e+02, -8.76077746e+02, -7.06871963e+01,
       -2.89317890e+02, -2.12975763e+03,  4.10384560e+01, -2.52474753e+02,
        2.06370598e+04, -7.84278020e+02, -1.03012333e+03, -1.49813592e+03,
        2.17536155e+02, -8.71367859e+02, -1.09119044e+04, -8.52324681e+03,
       -1.57390101e+03, -

# [Test] Loss Function

In [30]:
def objective_function(V, x_c_l, y_c_l, z_c_l, x_c_r, y_c_r, z_c_r,
                      cmd_lnt, cmd_lnp, cmd_unt, cmd_et, cmd_lep, cmd_rep):

    # Convert to Tensor
    x_c_l = torch.Tensor(x_c_l).reshape(-1,1).to(dtype=dtype, device=device)
    y_c_l = torch.Tensor(y_c_l).reshape(-1,1).to(dtype=dtype, device=device)
    z_c_l = torch.Tensor(z_c_l).reshape(-1,1).to(dtype=dtype, device=device)
    x_c_r = torch.Tensor(x_c_r).reshape(-1,1).to(dtype=dtype, device=device)
    y_c_r = torch.Tensor(y_c_r).reshape(-1,1).to(dtype=dtype, device=device)
    z_c_r = torch.Tensor(z_c_r).reshape(-1,1).to(dtype=dtype, device=device)
    cmd_lnt = torch.Tensor(cmd_lnt).reshape(-1,1).to(dtype=dtype, device=device)
    cmd_lnp = torch.Tensor(cmd_lnp).reshape(-1,1).to(dtype=dtype, device=device)
    cmd_unt = torch.Tensor(cmd_unt).reshape(-1,1).to(dtype=dtype, device=device)
    cmd_et = torch.Tensor(cmd_et).reshape(-1,1).to(dtype=dtype, device=device)
    cmd_lep = torch.Tensor(cmd_lep).reshape(-1,1).to(dtype=dtype, device=device)
    cmd_rep = torch.Tensor(cmd_rep).reshape(-1,1).to(dtype=dtype, device=device)
    
    #Joint-to-Motor Polynomial Models
    neck_pitch = (torch.cat((torch.ones(cmd_lnt.shape[0], 1), cmd_lnt, cmd_lnt.pow(2), cmd_lnt.pow(3)), dim=1) @ torch.Tensor([[V[0]],[V[1]],[V[2]],[V[3]]]).to(dtype=dtype, device=device))
    neck_yaw = (torch.cat((torch.ones(cmd_lnp.shape[0], 1), cmd_lnp, cmd_lnp.pow(2), cmd_lnp.pow(3)), dim=1) @ torch.Tensor([[V[4]],[V[5]],[V[6]],[V[7]]]).to(dtype=dtype, device=device))
    head_pitch = (torch.cat((torch.ones(cmd_unt.shape[0], 1), cmd_unt, cmd_unt.pow(2), cmd_unt.pow(3)), dim=1) @ torch.Tensor([[V[8]],[V[9]],[V[10]],[V[11]]]).to(dtype=dtype, device=device))
    eyes_pitch = (torch.cat((torch.ones(cmd_et.shape[0], 1), cmd_et, cmd_et.pow(2), cmd_et.pow(3)), dim=1) @ torch.Tensor([[V[12]],[V[13]],[V[14]],[V[15]]]).to(dtype=dtype, device=device))
    lefteye_yaw = (torch.cat((torch.ones(cmd_lep.shape[0], 1), cmd_lep, cmd_lep.pow(2), cmd_lep.pow(3)), dim=1) @ torch.Tensor([[V[16]],[V[17]],[V[18]],[V[19]]]).to(dtype=dtype, device=device))
    righteye_yaw = (torch.cat((torch.ones(cmd_rep.shape[0], 1), cmd_rep, cmd_rep.pow(2), cmd_rep.pow(3)), dim=1) @ torch.Tensor([[V[20]],[V[21]],[V[22]],[V[23]]]).to(dtype=dtype, device=device))

    # URDF Variable Assignment
    for joint in robot.joints:
        if joint.name == 'torso':
            joint.origin.position[0] = V[24]
            joint.origin.position[1] = V[25]
            joint.origin.position[2] = V[26]
        elif joint.name == 'neck_pitch':
            joint.origin.rotation[0] = V[27]
        elif joint.name == 'head_pitch':
            joint.origin.position[2] = V[28]
            joint.origin.rotation[0] = V[29]
            joint.origin.rotation[2] = V[30]
        elif joint.name == 'eyes_pitch':
            joint.origin.position[0] = V[31]
            joint.origin.position[2] = V[32]
            joint.origin.rotation[0] = V[33]
            joint.origin.rotation[2] = V[34]
        elif joint.name == 'lefteye_yaw':
            joint.origin.position[1] = V[35]
        elif joint.name == 'righteye_yaw':
            joint.origin.position[1] = V[36]
        elif joint.name == 'lefteye_cam':
            joint.origin.position[0] = V[37]
            joint.origin.rotation[0] = V[38]
            joint.origin.rotation[1] = V[39]
            joint.origin.rotation[2] = V[40]
        elif joint.name == 'righteye_cam':
            joint.origin.position[0] = V[41]
            joint.origin.rotation[0] = V[42]
            joint.origin.rotation[1] = V[43]
            joint.origin.rotation[2] = V[44]
    
    # XML to String
    urdf_str = xml_to_str(robot)
    
    # Kinematic Chain
    chain = pk.build_chain_from_urdf(urdf_str)
    chain.to(dtype=dtype, device=device)
    
    # Specifying Joint Angles (radians)
    joint_cmd = torch.cat((neck_pitch, neck_yaw, head_pitch, eyes_pitch, lefteye_yaw, righteye_yaw), dim=1)
    
    # Forward Kinematics
    ret = chain.forward_kinematics(joint_cmd)
    
    # Realsense to Left Eye Camera (Pytorch) with Points
    T_clprime = torch.linalg.inv(ret['realsense'].get_matrix()) @ ret['leftcamera'].get_matrix()
    
    # Realsense to Right Eye Camera (Pytorch) with Points
    T_crprime = torch.linalg.inv(ret['realsense'].get_matrix()) @ ret['rightcamera'].get_matrix()
    
    # Extrinsic Matrix
    T_left_ext = T_world_opencv @ T_clprime @ T_opencv_world
    T_right_ext = T_world_opencv @ T_crprime @ T_opencv_world
    
    
    # Chest Camera OpenCV Orientation 3D Projection Points
    left_chest_pts = torch.cat((x_c_l, y_c_l, z_c_l,torch.ones(x_c_l.shape[0], 1)), dim=1).reshape(-1,4,1)
    right_chest_pts = torch.cat((x_c_r, y_c_r, z_c_r,torch.ones(x_c_r.shape[0], 1)), dim=1).reshape(-1,4,1)
    
    # Points Projection
    u_left, v_left = projectPoints(left_chest_pts, T_left_ext, left_cam_mtx, left_dist_coef)   
    u_right, v_right = projectPoints(right_chest_pts, T_right_ext, right_cam_mtx, right_dist_coef)

    # Clamping
    u_min_val = 0
    u_max_val = 639 
    v_min_val = 0
    v_max_val = 479
    # u_left = torch.clamp(u_left, min=u_min_val, max=u_max_val)
    # v_left = torch.clamp(v_left, min=v_min_val, max=v_max_val)
    # u_right = torch.clamp(u_right, min=u_min_val, max=u_max_val)
    # v_right = torch.clamp(v_right, min=v_min_val, max=v_max_val)
    
    # # Scaling
    # u_left = min_max_scaler(u_left, u_min_val, u_max_val)
    # v_left = min_max_scaler(v_left, v_min_val, v_max_val)
    # u_right = min_max_scaler(u_right, u_min_val, u_max_val)
    # v_right = min_max_scaler(v_right, v_min_val, v_max_val)
    
    # True Value
    true_u_left = gaze_ctrs['left_eye']['x_center']
    true_v_left = gaze_ctrs['left_eye']['y_center']
    true_u_right = gaze_ctrs['right_eye']['x_center']
    true_v_right = gaze_ctrs['right_eye']['y_center']
    # true_u_left = min_max_scaler(gaze_ctrs['left_eye']['x_center'], u_min_val, u_max_val)
    # true_v_left = min_max_scaler(gaze_ctrs['left_eye']['y_center'], v_min_val, v_max_val)
    # true_u_right = min_max_scaler(gaze_ctrs['right_eye']['x_center'], u_min_val, u_max_val)
    # true_v_right = min_max_scaler(gaze_ctrs['right_eye']['y_center'], v_min_val, v_max_val)

    
    # Loss
    residuals = torch.cat(((u_left-true_u_left).reshape(-1,1),(v_left-true_v_left).reshape(-1,1),(u_right-true_u_right).reshape(-1,1),(v_right-true_v_right).reshape(-1,1)), dim=1).cpu().numpy()
    res_sum = np.sum(residuals, axis=1)

    # Return Loss
    return res_sum

In [31]:
# Input

# Input
V = np.array(var_list)
x_c_l = data_df['x_c_l'].to_numpy()
y_c_l = data_df['y_c_l'].to_numpy()
z_c_l = data_df['z_c_l'].to_numpy()
x_c_r = data_df['x_c_r'].to_numpy()
y_c_r = data_df['y_c_r'].to_numpy()
z_c_r = data_df['z_c_r'].to_numpy()
cmd_lnt = data_df['cmd_theta_lower_neck_tilt'].to_numpy()
cmd_lnp = data_df['cmd_theta_lower_neck_pan'].to_numpy()
cmd_unt = data_df['cmd_theta_upper_neck_tilt'].to_numpy()
cmd_et = data_df['cmd_theta_eyes_tilt'].to_numpy()
cmd_lep = data_df['cmd_theta_left_eye_pan'].to_numpy()
cmd_rep = data_df['cmd_theta_right_eye_pan'].to_numpy()

In [32]:
# Test Objective Function
objective_function(V, x_c_l, y_c_l, z_c_l, x_c_r, y_c_r, z_c_r,
                      cmd_lnt, cmd_lnp, cmd_unt, cmd_et, cmd_lep, cmd_rep)

array([-1.08486150e+03, -1.25741384e+05, -1.12664040e+03, -1.51669514e+03,
        2.97465678e+03, -4.90070152e+02, -1.71654909e+03, -1.41960215e+02,
       -1.24544292e+03, -1.01025342e+03, -7.63595710e+02,  3.80876493e+02,
       -2.43649136e+04, -5.29582200e+02, -6.05910583e+02, -1.54421272e+03,
       -5.25724737e+02, -1.31320837e+03, -2.98135049e+02, -2.04729498e+02,
       -1.74388918e+04, -5.79856738e+02, -4.33598657e+03, -2.86912430e+03,
       -3.62537404e+03, -1.05561698e+03, -1.52276681e+03, -7.27690426e+02,
       -1.42825339e+03, -1.38634470e+03, -1.73136327e+03,  1.28379326e+03,
       -1.69045752e+02, -8.35826772e+02, -4.94382546e+02, -7.25348270e+02,
       -1.47991915e+03, -3.20844036e+02, -8.76077746e+02, -7.06871963e+01,
       -2.89317890e+02, -2.12975763e+03,  4.10384560e+01, -2.52474753e+02,
        2.06370598e+04, -7.84278020e+02, -1.03012333e+03, -1.49813592e+03,
        2.17536155e+02, -8.71367859e+02, -1.09119044e+04, -8.52324681e+03,
       -1.57390101e+03, -

# Least Squares

In [34]:
opt = least_squares(objective_function, V, verbose=2, args=(x_c_l, y_c_l, z_c_l, x_c_r, y_c_r, z_c_r,
                                                 cmd_lnt, cmd_lnp, cmd_unt, cmd_et, cmd_lep, cmd_rep))
opt

   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         4.5266e+12                                    7.76e+14    
       1              2         6.7360e+11      3.85e+12       4.11e-01       1.42e+14    
       2              3         1.8648e+11      4.87e+11       8.21e-01       3.00e+13    
       3              4         4.9326e+08      1.86e+11       8.21e-01       2.94e+10    
       4              7         3.9219e+08      1.01e+08       1.03e-01       1.96e+10    
       5              8         1.5039e+08      2.42e+08       2.57e-02       2.91e+09    
       6              9         8.5179e+07      6.52e+07       5.13e-02       2.94e+08    
       7             10         6.0823e+07      2.44e+07       1.03e-01       1.49e+08    
       8             11         3.2802e+07      2.80e+07       2.05e-01       6.27e+07    
       9             12         7.6021e+06      2.52e+07       4.11e-01       1.21e+08    

     message: The maximum number of function evaluations is exceeded.
     success: False
      status: 0
         fun: [-2.973e+00 -7.026e+01 ... -2.355e+01  3.920e+00]
           x: [ 1.499e+00  2.682e-01 ...  2.208e-01 -3.309e-01]
        cost: 156217.5452027244
         jac: [[-1.054e+03 -5.521e+02 ...  0.000e+00 -0.000e+00]
               [-1.149e+03  0.000e+00 ...  0.000e+00 -0.000e+00]
               ...
               [-1.124e+03  1.961e+02 ...  0.000e+00 -0.000e+00]
               [-1.057e+03 -3.688e+02 ...  0.000e+00 -0.000e+00]]
        grad: [-5.457e+00 -1.603e+00 ...  0.000e+00  0.000e+00]
  optimality: 25.25882834523145
 active_mask: [ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
        nfev: 4500
        njev: 4469

In [35]:
dict(zip(var_names_list, opt.x))

{'neck_pitch_polyfit_b0': 1.4991542142830776,
 'neck_pitch_polyfit_b1': 0.2682155719835249,
 'neck_pitch_polyfit_b2': 0.009998129616212811,
 'neck_pitch_polyfit_b3': -0.021193800094220058,
 'neck_yaw_polyfit_b0': 0.17057822809034867,
 'neck_yaw_polyfit_b1': 0.14545006309919084,
 'neck_yaw_polyfit_b2': -0.06511193312980075,
 'neck_yaw_polyfit_b3': 0.05967197627313077,
 'head_pitch_polyfit_b0': -2.194440028900599,
 'head_pitch_polyfit_b1': -0.3222688500413232,
 'head_pitch_polyfit_b2': 0.0016949567623989492,
 'head_pitch_polyfit_b3': 0.04537307475465156,
 'eyes_pitch_polyfit_b0': 0.7322716820530187,
 'eyes_pitch_polyfit_b1': -0.30665697683595167,
 'eyes_pitch_polyfit_b2': 0.07927760181560158,
 'eyes_pitch_polyfit_b3': 0.10351636615713994,
 'lefteye_yaw_polyfit_b0': -0.14181615680607512,
 'lefteye_yaw_polyfit_b1': 0.0935755945014784,
 'lefteye_yaw_polyfit_b2': -0.345783344809435,
 'lefteye_yaw_polyfit_b3': 1.5170466217794711,
 'righteye_yaw_polyfit_b0': 0.11195354446009034,
 'righteye_yaw

In [36]:
# URDF
W = opt.x

In [37]:
for joint in robot.joints:
    if joint.name == 'torso':
        joint.origin.position[0] = W[24]
        joint.origin.position[1] = W[25]
        joint.origin.position[2] = W[26]
    elif joint.name == 'neck_pitch':
        joint.origin.rotation[0] = W[27]
    elif joint.name == 'head_pitch':
        joint.origin.position[2] = W[28]
        joint.origin.rotation[0] = W[29]
        joint.origin.rotation[2] = W[30]
    elif joint.name == 'eyes_pitch':
        joint.origin.position[0] = W[31]
        joint.origin.position[2] = W[32]
        joint.origin.rotation[0] = W[33]
        joint.origin.rotation[2] = W[34]
    elif joint.name == 'lefteye_yaw':
        joint.origin.position[1] = W[35]
    elif joint.name == 'righteye_yaw':
        joint.origin.position[1] = W[36]
    elif joint.name == 'lefteye_cam':
        joint.origin.position[0] = W[37]
        joint.origin.rotation[0] = W[38]
        joint.origin.rotation[1] = W[39]
        joint.origin.rotation[2] = W[40]
    elif joint.name == 'righteye_cam':
        joint.origin.position[0] = W[41]
        joint.origin.rotation[0] = W[42]
        joint.origin.rotation[1] = W[43]
        joint.origin.rotation[2] = W[44]

In [40]:
urdf_path = os.path.join(os.getcwd(), 'urdf', 'chest_grace_results.urdf')

In [43]:
temp_str = robot.to_xml_string()
print(temp_str)

<Element 'robot' at 0x7f1a3893a360>